In [2]:
import numpy as np
import pandas as pd
import os

/tmp/ipykernel_222702/2443873611.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
patterns = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/imagen/patterns/imagen_patterns.csv', usecols=['Subject', 'PAT_R', 'PAT_L'])

In [7]:
set(patterns['PAT_R'])

{'??????', 'A', 'B', 'BC', 'BC ', 'BCD', 'BD', 'C', 'CA', 'CD', 'D', 'DC', nan}

In [8]:
set(patterns['PAT_L'])

{'??????', 'A', 'B', 'BC', 'BCD', 'BD', 'C', 'CD', 'D', nan}

In [14]:
# remove Nan
patterns.loc[patterns.isna()['PAT_R'] | patterns.isna()['PAT_L']]

,Subject,PAT_R,PAT_L
225,11019827,NaN,NaN
347,16908046,NaN,NaN


In [15]:
patterns = patterns.drop(index=[225, 347])

In [18]:
# remove ??????
patterns.loc[(patterns['PAT_L']=='??????') | (patterns['PAT_R']=='??????')]

,Subject,PAT_R,PAT_L
436,21218872,??????,??????


In [19]:
patterns = patterns.drop(index=[436])

In [22]:
## proportion of each pattern
# L
print(np.unique(patterns['PAT_L'], return_counts=True))
# R
print(np.unique(patterns['PAT_R'], return_counts=True))

(array(['A', 'B', 'BC', 'BCD', 'BD', 'C', 'CD', 'D'], dtype=object), array([364, 371, 311,  85,  41, 747,  56,  30]))
(array(['A', 'B', 'BC', 'BC ', 'BCD', 'BD', 'C', 'CA', 'CD', 'D', 'DC'],
      dtype=object), array([727, 524,  93,   1,  47,  54, 453,   1,  31,  73,   1]))


# define the 3 interruptions
types are non exclusive

In [26]:
# 1st (RS-CS)
patterns.loc[patterns['PAT_L'].isin(['B', 'BC', 'BD', 'BCD'])]

,Subject,PAT_R,PAT_L
0,1274,B,B
11,458976,CD,BC
21,820294,C,BCD
22,829055,B,BC
25,942170,BCD,BCD
...,...,...,...
1996,99559419,C,BCD
1997,99561404,BD,BC
2003,99873252,B,BC
2005,99888850,D,BD


In [27]:
patterns.loc[patterns['PAT_R'].isin(['B', 'BC', 'BD', 'BCD'])]

,Subject,PAT_R,PAT_L
0,1274,B,B
1,22453,B,A
5,112288,B,A
16,613223,B,C
22,829055,B,BC
...,...,...,...
1994,99529479,B,B
1997,99561404,BD,BC
2002,99747799,BC,A
2003,99873252,B,BC


In [28]:
# 2nd (CS-OTS)
patterns.loc[patterns['PAT_L'].isin(['C', 'BC', 'CD', 'BCD'])]

,Subject,PAT_R,PAT_L
3,106601,A,C
4,106871,A,C
6,215284,C,CD
7,240546,C,C
8,297685,C,C
...,...,...,...
2000,99616225,C,CD
2001,99677574,A,C
2003,99873252,B,BC
2004,99875982,C,C


In [29]:
patterns.loc[patterns['PAT_R'].isin(['C', 'BC', 'CD', 'BCD'])]

,Subject,PAT_R,PAT_L
6,215284,C,CD
7,240546,C,C
8,297685,C,C
10,397377,C,C
11,458976,CD,BC
...,...,...,...
1996,99559419,C,BCD
1998,99580654,C,A
2000,99616225,C,CD
2002,99747799,BC,A


In [30]:
# 3rd (RS-OTS)
patterns.loc[patterns['PAT_L'].isin(['D', 'CD', 'BD', 'BCD'])]

,Subject,PAT_R,PAT_L
6,215284,C,CD
21,820294,C,BCD
25,942170,BCD,BCD
29,1121275,B,BCD
35,1330914,C,BD
...,...,...,...
1971,98537667,BD,BD
1996,99559419,C,BCD
2000,99616225,C,CD
2005,99888850,D,BD


In [31]:
patterns.loc[patterns['PAT_R'].isin(['D', 'CD', 'BD', 'BCD'])]

,Subject,PAT_R,PAT_L
11,458976,CD,BC
25,942170,BCD,BCD
33,1298929,BD,BC
36,1338177,D,C
49,1781386,BD,BCD
...,...,...,...
1980,98847514,D,C
1981,98929068,BD,B
1982,98958194,BD,C
1997,99561404,BD,BC


3rd interruption is more rare

# save individual interruption CSVs

In [44]:
save_dir = '/neurospin/dico/data/deep_folding/current/datasets/imagen/patterns/'

In [45]:
# 1st (RS-CS)
list_patterns = ['B', 'BC', 'BD', 'BCD']
name = 'RS_CS'
interrup_L = patterns['PAT_L'].isin(list_patterns).astype(int).tolist()
interrup_R = patterns['PAT_R'].isin(list_patterns).astype(int).tolist()
interrup = pd.DataFrame({'Subject': patterns.Subject, f'Right_Interrup_{name}': interrup_R, f'Left_Interrup_{name}': interrup_L})
interrup.to_csv(os.path.join(save_dir,f'Interruption_{name}.csv'), index=False)

In [48]:
# 2nd
list_patterns = ['C', 'BC', 'CD', 'BCD']
name = 'CS_OTS'
interrup_L = patterns['PAT_L'].isin(list_patterns).astype(int).tolist()
interrup_R = patterns['PAT_R'].isin(list_patterns).astype(int).tolist()
interrup = pd.DataFrame({'Subject': patterns.Subject, f'Right_Interrup_{name}': interrup_R, f'Left_Interrup_{name}': interrup_L})
interrup.to_csv(os.path.join(save_dir,f'Interruption_{name}.csv'), index=False)

In [49]:
# 3rd
list_patterns = ['D', 'CD', 'BD', 'BCD']
name = 'RS_OTS'
interrup_L = patterns['PAT_L'].isin(list_patterns).astype(int).tolist()
interrup_R = patterns['PAT_R'].isin(list_patterns).astype(int).tolist()
interrup = pd.DataFrame({'Subject': patterns.Subject, f'Right_Interrup_{name}': interrup_R, f'Left_Interrup_{name}': interrup_L})
interrup.to_csv(os.path.join(save_dir,f'Interruption_{name}.csv'), index=False)